In [1]:
import pandas as pd
import tensorflow as tf

### Config

In [2]:
train_meta_data = '../train.csv'
train_data_dir = '../input/paddy-disease-classification/train_images'
epochs = 100
lr = 1e-3
valid_split = 0.2
input_size=224
batch_size = 32
initializer = tf.keras.initializers.HeUniform()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=10,
                                              monitor='val_loss',
                                              restore_best_weights=True)